# Lecture 11: Data analysis with ```pandas```

In [ ]:
import pandas as pd
import numpy as np
import csv

import matplotlib.pyplot as plt

In the Lecture 5, you have already looked into ```csv.reader``` which converts data from CSV files into python strings. Working with this type of data is not always convenient. 


## Recap from the previous lectures

### ```csv``` library

### Isotope database

As an example, we will work with ```isotop_data.csv``` that was generated from https://www-nds.iaea.org/relnsd/vcharthtml/VChartHTML.html

This database contains information on physical properties of most of the known isotops (but not all!)

In [ ]:
with open("isotop_data.csv",'r') as dest_f:
    csv_reader = csv.reader(dest_f,
                           delimiter = ',',
                           quotechar = '"')
    data = [data for data in csv_reader]
    
data_array = np.asarray(data)

In [ ]:
data_array

### ```numpy``` library

In [ ]:
np.genfromtxt('isotop_data.csv', delimiter=',',invalid_raise=False)

## Pandas
 - library for data manipulation and analysis
 - imports data from CSV, JSON, SQL, Excel etc.
 - built on top of ```numpy```
 - most machine learning tools support ```pandas``` $\rightarrow$ used for data preprocessing in machine learning
 - optimized performance for big tables

## Reading the data file

In [ ]:
df = pd.read_csv("isotop_data.csv")

If you already know the content of the file and you don't need all the columns, you can read the file more efficiently with 

```usecols = ['column1', 'column2', 'column3']```

In [ ]:
df

In [ ]:
type(df)

In [ ]:
df['Z']

In [ ]:
type(df['Z'])

### What is inside?

In [ ]:
print(df.info())

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.columns

## Selecting data from the table

### Selecting columns

In [ ]:
df.loc[:, "N"]

In [ ]:
df.iloc[:, 1]

In [ ]:
df[["Z","N"]]

### Selecting rows

In [ ]:
df.loc[2:3]

In [ ]:
df.iloc[2:3]

### Selecting a range from the table

In [ ]:
df.loc[2:5, ["Z","N","symbol" ,"radius [fm]","half_life"]]

In [ ]:
df.iloc[2:6, [0,1,2,3,12]]

### Selecting by condition

In [ ]:
df[df['half_life']=='STABLE'] #selecting only stable isotops

In [ ]:
df[df['symbol']=='Ra'][['symbol', 'radius [fm]',' half_life [s]']] # selecting only Ra isotops

In [ ]:
df[(df['half_life']=='STABLE')&(df['N']>120)] # selecting isotops that have more than 120 neutrons



__Caution!__ Comparing values in ```pandas``` series works the same as in numpy arrays. For example, logical operator ```and``` can compare only two single values and cannot do element-wise comparison of the arrays. You can use ```numpy.logical_and``` or ```&``` for 1d Boolean arrays, but better use ```numpy.logical_and``` for more complex structures.


In [ ]:
df_test[np.logical_and(df_test['radius [fm]']>1,pd.to_numeric(df_test['half_life'])>1)]

In [ ]:
df_test[(df_test['radius [fm]']>1)|(pd.to_numeric(df_test['half_life'])>1)]

## Manipulating the dataframe - saving into new file, adding entries, renaming

In [ ]:
df.columns

In [ ]:
df_reduced = df[['Z', 'N', 'symbol', 'radius [fm]',' abundance [mole fract.]',' half_life [s]','atomic mass [AMU * 1.E-3 ]',' discovery']]

# please note, that doing df = df[["column1", "column2", ...]] is a bad style that leads to many possible errors!

In [ ]:
df_reduced

### Rename columns

In [ ]:
df_reduced.rename({'radius [fm]': 'R_fm', ' abundance [mole fract.]': 'abund.',' half_life [s]':'tau', 'atomic mass [AMU * 1.E-3 ]':'A', ' discovery':'year'}, axis=1)

```df.raname``` does not modify the original ```df```

In [ ]:
df_reduced = df_reduced.rename({'radius [fm]': 'R_fm', ' abundance [mole fract.]': 'abund.',' half_life [s]':'tau', 'atomic mass [AMU * 1.E-3 ]':'A', ' discovery':'year'}, axis=1)

In [ ]:
df_reduced['A'] = df_reduced['A']/1e6

In [ ]:
df_reduced

### Add/delete a column or raw  

In [ ]:
df_reduced['stability']=df_reduced['tau'].isnull()

In [ ]:
df_reduced

In [ ]:
df_reduced['stability2']=df_reduced['tau'].isnull()

In [ ]:
# adding a row of ones at the end of the table
df_reduced.loc[df_reduced.shape[0]] = np.ones(df_reduced.shape[1])

In [ ]:
df_reduced

In [ ]:
# deleting the recentrly added row and column

df_reduced=df_reduced.drop('stability2',axis=1)
df_reduced=df_reduced.drop([df_reduced.shape[0]-1])

In [ ]:
df_reduced

### Saving into a file

In [ ]:
df_stable = df_reduced[df_reduced['stability']==1]
df_stable.to_csv("stable_isotops.csv")


df_unstable = df_reduced[df_reduced['stability']==0]
df_unstable.to_csv("unstable_isotops.csv")


## Quick analysis tools

### Histogram

In [ ]:
df_stable["year"].hist()

In [ ]:
df_unstable["year"].hist()

In [ ]:
# same plot, combine with matplotlib functions

plt.hist(df_stable["year"], bins=20)
plt.xlabel("Year")
plt.ylabel ("Number of discovered stable isotops")

### Scatter plot

In [ ]:
df_stable[["R_fm", "A"]].plot.scatter("A", "R_fm")

In [ ]:
df_unstable[["R_fm", "A"]].plot.scatter("A", "R_fm")
# plt.plot(df_unstable["A"],1.2*df_unstable['A']**(1/3))
# plt.xlim([0,50])

### Bar plot

In [ ]:
df_stable[df_stable['symbol']=='Fe']

df_stable[df_stable['symbol']=='Fe'].plot.bar("N","abund.")

### Example 1: finding the most long-living Radium isotop

In [ ]:
df_unstable.describe()

In [ ]:
df_radium = df_unstable[df_unstable['symbol']=='Ra']

In [ ]:
df_radium.loc[df_radium['tau'].idxmax()]

### Example 2: Finding only unstable elements

In [ ]:
df_unstable[~df_unstable['symbol'].isin(df_stable['symbol'])]

### Example 3: ten lagest nuclei 

In [ ]:
df_unstable.sort_values(by='R_fm',na_position='first')

In [ ]:
df_unstable.sort_values(by='R_fm',na_position='first').iloc[-10:]